In [2]:
from delta.tables import DeltaTable

# Διαβάζουμε το Gold table από το Lakehouse
gold_df = spark.read.table("gold_sales")

# Όνομα πίνακα στο Warehouse
warehouse_table_name = "WH_GoldSales"

# Αν υπάρχει ήδη στο Warehouse → MERGE
if spark.catalog.tableExists(warehouse_table_name):
    warehouse_table = DeltaTable.forName(spark, warehouse_table_name)
    
    warehouse_table.alias("target") \
        .merge(
            gold_df.alias("source"),
            "target.SaleID = source.SaleID"
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()

# Αν δεν υπάρχει → δημιουργία πίνακα
else:
    gold_df.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(warehouse_table_name)



StatementMeta(, 09dacb73-bf0c-4cee-8f80-36883dbb6385, 4, Finished, Available, Finished)